### Night Seven assignment - import HL7 message and convert to JSON 

Conceptually, this assigment requires the following:  
1. Read in the HL7 messages.
2. Create an object capable of accomplishing the following:
        a. Use the Segment identifiers as "Keys"     
        b. Use the Fields as "Values"
        c. Use Components with sub-components as "sub-Values"

In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import csv
import glob
import io
from xlsxwriter.workbook import Workbook
from UserDict import UserDict
import string
import json

In [2]:
pd.set_option('display.mpl_style','default')


Simple tuple creation - to see the results.

In [3]:
tuple_1 = ('name1', 1, 'sample1', '^', '|')
tuple_2 = ('name2', 2, 'sample2')
tuple_3 = tuple_1 + tuple_2
print tuple_3

('name1', 1, 'sample1', '^', '|', 'name2', 2, 'sample2')


Below - usually a good idea to read a file (or files) before importing it (them), to make sure it (they) contain(s) data, that the data is not corrupted, etc.

In [4]:
pd.read_csv('C:\Users\Bill\msg1.txt')

,MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1
0,EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
1,PID|1||V000042610|M88604|BARTON^CLINTON^FRANCI...
2,NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
3,NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
4,PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|...
5,AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|M...
6,ACC|20150722^|11
7,GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L....
8,GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
9,IN1|1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^...


In [5]:
pd.read_csv('C:\Users\Bill\msg2.txt')

,MSH|^~\&||ABCDE|||201507221833||ADT^A08|CAGTADM.1.10532250|P|2.1
0,EVN|A08|201507221833|||R.HIM.CLV^VEALS^CHENEKA...
1,PID|1||V000303465|V256537|ROMANOVA^NATALIA^ALI...
2,PV1|1|E|V.OBED^^|EM|||STRANGE^STEPHEN^VINCENT^...
3,ACC|20141030^|10
4,GT1|1||ROMANOVA^NATALIA^ALIANOVA^^^^||40 page ...
5,GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
6,IN1|1|MDGA001||MEDICAID GA|PO BOX 105204^^TUCK...
7,ZCD|1|ETHNICITY^ETHNICITY^2
8,ZIN|1|SP|HP - UB04 CLAIMS|N||||N|||||MDGA001
9,ZCS|UEMP|UNEMPLOYED^^UNEMPLOYED^GA^30092|U|NON...


msg3.txt must have an error; would not read without including 'error_bad_lines=False'

In [64]:
pd.read_csv('C:\Users\Bill\msg3.txt', error_bad_lines=False)

Skipping line 4: expected 1 fields, saw 2
Skipping line 5: expected 1 fields, saw 3
Skipping line 6: expected 1 fields, saw 2



,MSH|^~\&||ABCDE|||201507230002||ADT^A03|CAGTADM.1.10532923|P|2.1
0,EVN|A03|201507230001|||MIDNT RUN^^^^^^
1,PID|1||V000027071|V24686|BANNER^ROBERT^BRUCE^^...
2,AL1|3|DA|F006004180^tramadol^TRAMADOL|SV|MAKES...
3,AL1|4|DA|F006004807^ondansetron^From ZOFRAN (A...
4,AL1|5|DA|F006004812^ketorolac^From TORADOL|SV|...
5,ACC|20150717^|11
6,GT1|1||BANNER^ROBERT^BRUCE^^^||31 simplement d...
7,GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
8,IN1|1|MACP051||MEDICARE A B|CLAIMS DEPT^PO B...
9,IN1|2|MDGA002||MEDICAID GA SECONDARY|PO BOX 10...


#### 1. Tried a straight read using '|' as the delimiter, and with no header.

Code used: pd.read_table('C:\Users\Bill\msg1.txt', sep='|', header=None)

Gave a long error message. Last line was key: CParserError: error tokenizing data. C error: Expected 12 fields in line 3, saw 20.

Below - progress from the following links:
http://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data    

In [127]:
pd.read_table('C:\Users\Bill\msg1.txt', header=None, sep='|', error_bad_lines=False)
#pd.read_table('C:\Users\Bill\msg1.txt', header=None, sep='|', na_values=['NULL']) - did not work to fill in values

Skipping line 3: expected 12 fields, saw 20
Skipping line 6: expected 12 fields, saw 45
Skipping line 9: expected 12 fields, saw 21
Skipping line 10: expected 12 fields, saw 17
Skipping line 11: expected 12 fields, saw 44
Skipping line 15: expected 12 fields, saw 14



,0,1,2,3,4,5,6,7,8,9,10,11
0,MSH,^~\&,NaN,ABCDE,NaN,NaN,201507230021,NaN,ADT^A08,CAGTADM.1.10532994,P,2.1
1,EVN,A08,201507230021,NaN,NaN,R.CA.NAH^HALL^NANCY^A^^^,NaN,NaN,NaN,NaN,NaN,NaN
2,NK1,1,POSTEN^BOBBY^^^^,OT,102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^...,(000)000-0000,(000)000-0000,NaN,NaN,NaN,NaN,NaN
3,NK1,2,POSTEN^BOBBY^^^^,OT,102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^...,(000)000-0000,(000)000-0000,NaN,NaN,NaN,NaN,NaN
4,AL1,1,DA,F000004181^tramadol HCl^From ULTRAM,MO,JERKING,20140525,NaN,NaN,NaN,NaN,NaN
5,ACC,20150722^,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ZMR,1,GH00426357~IL00806247~M000103533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ZCD,1,ETHNICITY^ETHNICITY^2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ZCD,2,Z.ADDLRACE^ADDITIONAL RACE^W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ZCS,DIS,DISABLED^^DISABLED^GA^30092,N,NONE,NONE,08625,NaN,NaN,NaN,NaN,NaN


Spent some time researching how to fill in the missing data with nulls, and then decided that was not the way to go, as I was not sure how I would take it back out. After considering difference options, including exporting the HL7 files to Excel workbooks to fill in missing values, I decided to create tuples, using the Excel dialect. I discovered that the '|' delimeter was not necessary for this step.

http://stackoverflow.com/questions/15616139/create-a-list-of-tuples-from-csv-file

In [97]:
tuple1 = (pd.read_table('C:\Users\Bill\msg1.txt', header=None, dialect='excel'))
tuple1

,0
0,MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTAD...
1,EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2,PID|1||V000042610|M88604|BARTON^CLINTON^FRANCI...
3,NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
4,NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
5,PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|...
6,AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|M...
7,ACC|20150722^|11
8,GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L....
9,GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^


In [10]:
tuple1.columns

Int64Index([0], dtype='int64')

In [82]:
print tuple1

                                                    0
0   MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTAD...
1     EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2   PID|1||V000042610|M88604|BARTON^CLINTON^FRANCI...
3   NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
4   NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
5   PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|...
6   AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|M...
7                                    ACC|20150722^|11
8   GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L....
9               GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
10  IN1|1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^...
11             ZMR|1|GH00426357~IL00806247~M000103533
12                        ZCD|1|ETHNICITY^ETHNICITY^2
13                 ZCD|2|Z.ADDLRACE^ADDITIONAL RACE^W
14         ZIN|1|SP|GHP|Y|20150723|PPI||N|||||MDGA001
15  ZCS|DIS|DISABLED^^DISABLED^GA^30092|N|NONE|NON...


Played around with trying creating lists as well. Could not figure out how to create two separate lists from either the HL7 text files (group designator to one list, the segments to another list), or from a tuple, to them merge into a dict. Later on, see that I resorted to cheating, by taking the original HL7 text file, and creating two new files, to create my two lists.

In [99]:
t1_list = [pd.read_table('C:\Users\Bill\msg1.txt', header=None)]
t1_list

[                                                    0
 0   MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTAD...
 1     EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
 2   PID|1||V000042610|M88604|BARTON^CLINTON^FRANCI...
 3   NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
 4   NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floa...
 5   PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|...
 6   AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|M...
 7                                    ACC|20150722^|11
 8   GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L....
 9               GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
 10  IN1|1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^...
 11             ZMR|1|GH00426357~IL00806247~M000103533
 12                        ZCD|1|ETHNICITY^ETHNICITY^2
 13                 ZCD|2|Z.ADDLRACE^ADDITIONAL RACE^W
 14         ZIN|1|SP|GHP|Y|20150723|PPI||N|||||MDGA001
 15  ZCS|DIS|DISABLED^^DISABLED^GA^30092|N|NONE|NON...]

Repeat of steps takes with msg1.txt

In [84]:
tuple2 = (pd.read_table('C:\Users\Bill\msg2.txt', header=None, dialect='excel'))
tuple2

,0
0,MSH|^~\&||ABCDE|||201507221833||ADT^A08|CAGTAD...
1,EVN|A08|201507221833|||R.HIM.CLV^VEALS^CHENEKA...
2,PID|1||V000303465|V256537|ROMANOVA^NATALIA^ALI...
3,PV1|1|E|V.OBED^^|EM|||STRANGE^STEPHEN^VINCENT^...
4,ACC|20141030^|10
5,GT1|1||ROMANOVA^NATALIA^ALIANOVA^^^^||40 page ...
6,GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
7,IN1|1|MDGA001||MEDICAID GA|PO BOX 105204^^TUCK...
8,ZCD|1|ETHNICITY^ETHNICITY^2
9,ZIN|1|SP|HP - UB04 CLAIMS|N||||N|||||MDGA001


In [13]:
tuple2.columns

Int64Index([0], dtype='int64')

In [85]:
print tuple2

                                                    0
0   MSH|^~\&||ABCDE|||201507221833||ADT^A08|CAGTAD...
1   EVN|A08|201507221833|||R.HIM.CLV^VEALS^CHENEKA...
2   PID|1||V000303465|V256537|ROMANOVA^NATALIA^ALI...
3   PV1|1|E|V.OBED^^|EM|||STRANGE^STEPHEN^VINCENT^...
4                                    ACC|20141030^|10
5   GT1|1||ROMANOVA^NATALIA^ALIANOVA^^^^||40 page ...
6               GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
7   IN1|1|MDGA001||MEDICAID GA|PO BOX 105204^^TUCK...
8                         ZCD|1|ETHNICITY^ETHNICITY^2
9        ZIN|1|SP|HP - UB04 CLAIMS|N||||N|||||MDGA001
10  ZCS|UEMP|UNEMPLOYED^^UNEMPLOYED^GA^30092|U|NON...


In [86]:
t2_list = [pd.read_table('C:\Users\Bill\msg2.txt', header=None)]
t2_list

[                                                    0
 0   MSH|^~\&||ABCDE|||201507221833||ADT^A08|CAGTAD...
 1   EVN|A08|201507221833|||R.HIM.CLV^VEALS^CHENEKA...
 2   PID|1||V000303465|V256537|ROMANOVA^NATALIA^ALI...
 3   PV1|1|E|V.OBED^^|EM|||STRANGE^STEPHEN^VINCENT^...
 4                                    ACC|20141030^|10
 5   GT1|1||ROMANOVA^NATALIA^ALIANOVA^^^^||40 page ...
 6               GT1|2||^^^^^||^^^^^^^^|||||||||||^^^^
 7   IN1|1|MDGA001||MEDICAID GA|PO BOX 105204^^TUCK...
 8                         ZCD|1|ETHNICITY^ETHNICITY^2
 9        ZIN|1|SP|HP - UB04 CLAIMS|N||||N|||||MDGA001
 10  ZCS|UEMP|UNEMPLOYED^^UNEMPLOYED^GA^30092|U|NON...]

####  This is where I started "cheating," by creating separate HL7 text files. One for the group, one for the segment. Imported each into it's own list, and also it's own tuple.

In [102]:
t1_grp_lst = [pd.read_csv('C:\Users\Bill\msg1_grp.txt', header=None)]
t1_grp_lst

[      0
 0   MSH
 1   EVN
 2   PID
 3   NK1
 4   NK1
 5   PV1
 6   AL1
 7   ACC
 8   GT1
 9   GT1
 10  IN1
 11  ZMR
 12  ZCD
 13  ZCD
 14  ZIN
 15  ZCS]

In [103]:
t1_grp_tup = (pd.read_csv('C:\Users\Bill\msg1_grp.txt', header=None))
t1_grp_tup

,0
0,MSH
1,EVN
2,PID
3,NK1
4,NK1
5,PV1
6,AL1
7,ACC
8,GT1
9,GT1


In [104]:
t1_seg_lst = [pd.read_csv('C:\Users\Bill\msg1_seg.txt', header=None)]
t1_seg_lst

[                                                    0
 0   ^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
 1         A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
 2   1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
 3   1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
 4   2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
 5   1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
 6   1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
 7                                        20150722^|11
 8   1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
 9                   2||^^^^^||^^^^^^^^|||||||||||^^^^
 10  1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^GA^3...
 11                 1|GH00426357~IL00806247~M000103533
 12                            1|ETHNICITY^ETHNICITY^2
 13                     2|Z.ADDLRACE^ADDITIONAL RACE^W
 14             1|SP|GHP|Y|20150723|PPI||N|||||MDGA001
 15  DIS|DISABLED^^DISABLED^GA^30092|N|NONE|NONE|08625]

In [125]:
t1_seg_tup = (pd.read_csv('C:\Users\Bill\msg1_seg.txt', header=None))
t1_seg_tup

,0
0,^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
1,A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2,1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
3,1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
4,2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
5,1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
6,1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
7,20150722^|11
8,1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
9,2||^^^^^||^^^^^^^^|||||||||||^^^^


Merged the two tuples into a single dataframe. http://stackoverflow.com/questions/21231834/creating-a-pandas-dataframe-from-columns-of-other-dataframes-with-similar-indexe

Step to see if I can create the dictionary for msg1.txt. Answer: Nope...

In [117]:
t1__lst_dict = dict(zip(t1_grp_lst, t1_seg_lst))
print t1__lst_dict

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

Merged the two lists into a single dataframe.

In [109]:
#t1_mrg_tup = pd.concat([t1_grp_tup, t1_seg_tup], axis=1, keys=['grp', 'seg'])
t1_mrg_tup = pd.concat([t1_grp_tup, t1_seg_tup], axis=1)
t1_mrg_tup

,0,0
0,MSH,^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
1,EVN,A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2,PID,1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
3,NK1,1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
4,NK1,2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
5,PV1,1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
6,AL1,1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
7,ACC,20150722^|11
8,GT1,1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
9,GT1,2||^^^^^||^^^^^^^^|||||||||||^^^^


In [121]:
t1_mrg_lst = pd.concat([t1_grp_tup, t1_seg_tup],axis=1)
t1_mrg_lst

,0,0
0,MSH,^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
1,EVN,A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2,PID,1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
3,NK1,1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
4,NK1,2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
5,PV1,1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
6,AL1,1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
7,ACC,20150722^|11
8,GT1,1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
9,GT1,2||^^^^^||^^^^^^^^|||||||||||^^^^


In [135]:
# Try coverting the dataframe created from two lists to JSON
asjson = json.dumps(t1_mrg_lst)
asjson

TypeError:       0                                                  0
0   MSH  ^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
1   EVN        A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2   PID  1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
3   NK1  1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
4   NK1  2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
5   PV1  1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
6   AL1  1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
7   ACC                                       20150722^|11
8   GT1  1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
9   GT1                  2||^^^^^||^^^^^^^^|||||||||||^^^^
10  IN1  1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^GA^3...
11  ZMR                 1|GH00426357~IL00806247~M000103533
12  ZCD                            1|ETHNICITY^ETHNICITY^2
13  ZCD                     2|Z.ADDLRACE^ADDITIONAL RACE^W
14  ZIN             1|SP|GHP|Y|20150723|PPI||N|||||MDGA001
15  ZCS  DIS|DISABLED^^DISABLED^GA^30092|N|NO

In [136]:
# Try coverting the dataframe created from two tuples to JSON
asjson = json.dumps(t1_mrg_tup)
asjson

TypeError:       0                                                  0
0   MSH  ^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1....
1   EVN        A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
2   PID  1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^...
3   NK1  1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
4   NK1  2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating...
5   PV1  1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SEL...
6   AL1  1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JE...
7   ACC                                       20150722^|11
8   GT1  1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Tr...
9   GT1                  2||^^^^^||^^^^^^^^|||||||||||^^^^
10  IN1  1|MDGA001||MEDICAID GA|PO BOX 5000^^MCRAE^GA^3...
11  ZMR                 1|GH00426357~IL00806247~M000103533
12  ZCD                            1|ETHNICITY^ETHNICITY^2
13  ZCD                     2|Z.ADDLRACE^ADDITIONAL RACE^W
14  ZIN             1|SP|GHP|Y|20150723|PPI||N|||||MDGA001
15  ZCS  DIS|DISABLED^^DISABLED^GA^30092|N|NO